# Part 1
## sp_exact_3 
This algorithm implements the exact algorithm for computing an optimal MSA of 3 sequences and its score

*(decribed on page 408 in [Ohlebusch], or in Section 14.6.1 in [Gusfield])*

In [12]:
!pip install biopython

In [13]:
### Main script used to run the programs ###
from Bio import SeqIO
import pandas as pd
import numpy as np



### INPUT PARAMETERS ###

matrix = pd.read_csv("scorematrix.txt", index_col = 0, header = 0)
substitution_matrix = matrix.to_dict("dict")

# Load sequences
seqs = []
for seqrecord in SeqIO.parse("brca1-testseqs.fasta", "fasta"): # Change name/path of files if needed
        seqs.append(seqrecord)


# set gapcost
gap = 5

#Labels
length=len(seqs)
names=[]
for i in range(0,length):
    names.append(seqs[i].id)
print(names)

['brca1_bos_taurus', 'brca1_canis_lupus', 'brca1_gallus_gallus', 'brca1_homo_sapiens', 'brca1_macaca_mulatta', 'brca1_mus_musculus', 'brca1_pan_troglodytes', 'brca1_rattus_norvegicus']


In [14]:
# Returns the score of an exact multiple alignment of three sequences
from Bio import SeqIO
import numpy as np
import pandas as pd

### INPUT PARAMETERS ###

matrix = pd.read_csv("scorematrix.txt", index_col = 0, header = 0)
substitution_matrix = matrix.to_dict("dict")

# set gapcost
gap = 5

### Main function ###

# SP-score
def SP(A, B, C, submatrix, gapcost):
        a = [A,B,C]

        for i in range(len(a)):
                if a[i] == "W":
                        a[i] = "A"
                elif a[i] == "S":
                        a[i] = "C"
                elif a[i] == "R":
                        a[i] = "G"
                elif a[i] == "N":
                        a[i] = "T"

        gaps = a.count("-")

        if gaps == 0:
                score = submatrix[a[0]][a[1]] + submatrix[a[1]][a[2]] + submatrix[a[0]][a[2]]
        elif gaps == 1:
                a.remove('-')
                score = submatrix[a[0]][a[1]] + gapcost + gapcost
        elif gaps == 2:
                score = gapcost + gapcost
        else:
                return "SP-error"
        return score

def sp_exact_3(seqs,subm,gapc, test = False):
        if test == True:
                A = seqs[0]
                B = seqs[1]
                C = seqs[2]
        else:
                A = seqs[0].seq
                B = seqs[1].seq
                C = seqs[2].seq
        A = A.upper()
        B = B.upper()
        C = C.upper()

        ### Test sequences ###
        if len(A) == 0:
                return "Error: Seq1 is empty"
        if len(B) == 0:
                return "Error: Seq2 is empty"
        if len(C) == 0:
                return "Error: Seq3 is empty"

        alpha = list(subm.keys())
        add_letters= ["W","S", "N", "R"]
        for i in add_letters:
                alpha.append(i)

        if all(ele in alpha for ele in A) == False:
                return "Error: Seq1 contains unknown character"
        if all(ele in alpha for ele in B) == False:
                return "Error: Seq2 contains unknown character"
        if all(ele in alpha for ele in C) == False:
                return "Error: Seq3 contains unknown character"

        T = np.zeros([len(A)+1,len(B)+1,len(C)+1])

        for i in range(len(A)+1):
                for j in range(len(B)+1):
                        for k in range(len(C)+1):
                                v0=v1=v2=v3=v4=v5=v6=v7 = float("inf")
                                if i == 0 and j == 0 and k == 0:
                                        v0 = 0
                                if i > 0 and j > 0 and k > 0:
                                        v1 = T[i-1,j-1,k-1] + SP(A[i-1], B[j-1], C[k-1], subm, gapc)
                                if i > 0 and j > 0 and k >=0:
                                        v2 = T[i-1,j-1,k] + SP(A[i-1], B[j-1], "-", subm, gapc)
                                if i > 0 and j >= 0 and k>0:
                                        v3 = T[i-1,j,k-1] + SP(A[i-1], "-", C[k-1], subm, gapc)
                                if i >= 0 and j>0 and k>0:
                                        v4 = T[i, j-1, k-1] + SP("-", B[j-1], C[k-1], subm, gapc)
                                if i >=0 and j >=0 and k>0:
                                        v5 = T[i, j, k-1] + SP("-", "-", C[k-1], subm, gapc)
                                if i >=0 and j > 0 and k>=0:
                                        v6 = T[i, j-1, k] + SP("-", B[j-1], "-", subm, gapc)
                                if i >0 and j >=0 and k>=0:
                                        v7 = T[i - 1, j, k] + SP(A[i-1], "-", "-", subm, gapc)
                                T[i,j,k] = min(v0,v1,v2,v3,v4,v5,v6,v7)
        return T[-1,-1,-1]



# Part 2
The second part of the project is to use your MSA programs (or any other MSA programs) to align the 8 sequences of length 200 in brca1-testseqs.fasta as close to optimum as possible 

-Dependencies

In [15]:
# Two-approximation alignment helper
# SP-score
def SPapprox(col, submatrix, gapcost):
        col_score = 0
        for i in range(1,len(col)+1):
                for j in range(i):
                        a = [col[i-1], col[j]] #index?
                        gaps = a.count("-")
                        if gaps == 0:
                                score = submatrix[a[0]][a[1]]
                        elif gaps == 1:
                                score = gapcost
                        elif gaps == 2:
                                score = 0
                        else:
                                return "SP-error"
                        col_score = col_score + score
        return col_score #score

In [16]:
# Two-approximation alignment helper

def AlignMatrix(seqs, sub_matrix, gapcost):
    table = np.zeros((len(seqs), len(seqs)))
    for j in range(len(seqs)):
        for i in range(len(seqs)):
            table[i, j] = LinearAlignment(seqs[j], seqs[i], sub_matrix, gapcost)
    rowsum = table.sum(axis=1)
    minval = rowsum.argmin()
    return table, minval

In [17]:
# Two-approximation alignment helpers
import numpy as np

def BacktrackLinear(table, i, j, seq1, seq2, sub_matrix, gapcost, test):
    align1 = []
    align2 = []

    if test == False:
        si = seq1
        sj = seq2
    else:
        si = seq1
        sj = seq2

    ### Backtrack ###
    while i > 0 or j > 0:
        if si[i - 1] == "W":
            si_char = "A"
        elif si[i - 1] == "S":
            si_char = "C"
        elif si[i - 1] == "R":
            si_char = "G"
        elif si[i - 1] == "N":
            si_char = "T"
        else:
            si_char = si[i - 1]

        if sj[j - 1] == "W":
            sj_char = "A"
        elif sj[j - 1] == "S":
            sj_char = "C"
        elif sj[j - 1] == "R":
            sj_char = "G"
        elif sj[j - 1] == "N":
            sj_char = "T"
        else:
            sj_char = sj[j - 1]

        substitution = table[i - 1, j - 1] + sub_matrix[si_char][sj_char]
        if i > 0 and j > 0 and substitution == table[i, j]:
            align1.append(si_char)
            align2.append(sj_char)
            i = i - 1
            j = j - 1
        elif i >= 0 and j > 0 and table[i, j - 1] + gapcost == table[i, j]:
            align1.append("-")
            align2.append(sj_char)
            j = j - 1
        elif i > 0 and j >= 0 and table[i - 1, j] + gapcost == table[i, j]:
            align1.append(si_char)
            align2.append("-")
            i = i - 1
        else:
            return "error"
    ## Make pretty print of alingment ##
    align1.reverse()
    align2.reverse()
    res = []
    for i in range(len(align1)):
        res.append([align1[i],align2[i]])
    return res

def LinearAlignment(seq1, seq2, sub_matrix, gapcost, backtrack=False, test=False):
    ### Test sequences ###
    if len(seq1) == 0:
        return "Error: Seq1 is empty"
    if len(seq2) == 0:
        return "Error: Seq2 is empty"

    if test == False:
        si = seq1.upper()
        sj = seq2.upper()

    else:
        si = seq1.upper()
        sj = seq2.upper()

    ###   Initialize table. seq 1 is y axis (j) and seq 2 is x axis (i)    ###
    table = np.zeros([len(si) + 1, len(sj) + 1])
    for i in range(1, len(si) + 1):
        table[i, 0] = gapcost * i
    for j in range(1, len(sj) + 1):
        table[0, j] = gapcost * j

    ###   Fill out table    ###
    for i in range(1, len(si) + 1):
        if si[i - 1] == "W":
            si_char = "A"
        elif si[i - 1] == "S":
            si_char = "C"
        elif si[i - 1] == "R":
            si_char = "G"
        elif si[i - 1] == "N":
            si_char = "T"
        else:
            si_char = si[i - 1]

        for j in range(1, len(sj) + 1):
            if sj[j - 1] == "W":
                sj_char = "A"
            elif sj[j - 1] == "S":
                sj_char = "C"
            elif sj[j - 1] == "R":
                sj_char = "G"
            elif sj[j - 1] == "N":
                sj_char = "T"
            else:
                sj_char = sj[j - 1]



            ### Recursive formula ###
            substitution = table[i - 1, j - 1] + sub_matrix[si_char][sj_char]
            insertion = table[i, j - 1] + gapcost
            deletion = table[i - 1, j] + gapcost
            table[i, j] = (min(substitution, insertion, deletion))

    if backtrack == True:
        res = BacktrackLinear(table, i, j, seq1, seq2, sub_matrix, gapcost, test)
        if test == False:
            return res
        return "Score:{}".format(table[i, j])

    return table[i, j]



In [18]:
# Two-approximation alignment helper
# #Mi = “Mi-1 extended with A”
def ExtendM(M,A):
    MA = []  # We open an empty matrix to store the extension
    i = 0  # We initialize a pointer for M columns
    j = 0  # We initialize a pointer for A columns

    while i < len(M) and j < len(A):

        if M[i][0] == '-' and A[j][0] == "-":
            M[i].append(A[j][1])
            MA.append(M[i])
            i = i + 1
            j = j + 1

        elif M[i][0] == '-' and A[j][0] != "-":
            M[i].append('-')
            MA.append(M[i])
            i = i + 1

        elif M[i][0] != '-' and A[j][0] == '-':
            c = ['-'] * len(M[i])
            c.append(A[j][1])
            MA.append(c)
            j = j + 1

        elif M[i][0] != '-' and A[j][0] != '-':
            M[i].append(A[j][1])
            MA.append(M[i])
            i = i + 1
            j = j + 1

    if i < len(M):
        while i < len(M):
            MA.append(M[i])
            i = i + 1

    return MA

-The MSA code

In [19]:
# Returns the score of a two-approximation multiple alignment of k sequences
from Bio import SeqIO
import numpy as np
import pandas as pd

### INPUT PARAMETERS ###

matrix = pd.read_csv("scorematrix.txt", index_col = 0, header = 0)
substitution_matrix = matrix.to_dict("dict")



# set gapcost
gap = 5

### Main function ###
def sp_approx(seqs, sub_matrix, gapcost, get_alignment = False, test = False, get_center = False):
    if test == False:
        seqs1 = []
        for s in seqs:
            seqs1.append(s.seq)
        seqs = seqs1

    alpha = list(sub_matrix.keys())
    add_letters = ["W", "S", "N", "R"]
    for i in add_letters:
        alpha.append(i)

    ## Testing ##
    for str in seqs:
        str = str.upper()
        if len(str) == 0:
            return "Error: One or more sequence is empty"

        if all(ele in alpha for ele in str) == False:
                return "Error: File contains unknown character"


    matrix, minval = AlignMatrix(seqs, sub_matrix, gapcost)
    if get_center == True:
        print("Index of centerstring (pythonic index):", minval)
    k = len(matrix)
    M = []
    total = 0
    for i in range(len(seqs[minval])):
        M.append([seqs[minval][i]])

    for i in range(0, k):  # +1`?
        if i == minval:
            pass
        else:
            A = LinearAlignment(seqs[minval], seqs[i], sub_matrix, gapcost, backtrack=True)
            M = ExtendM(M, A)
    for col in M:
        score = SPapprox(col, sub_matrix, gapcost)
        total = total + score
    if get_alignment == True:
        arr1 = np.array(M)
        arr1_transpose = arr1.transpose()
        test=pd.DataFrame(data=arr1_transpose)
        test.index = names
        test.to_text(r'pandassssttsssss.txt', header=True, index=True)
        print(test)
    return total




In [20]:
## Print out result ##
print("Alignment score:", sp_approx(seqs, substitution_matrix, gap, get_center = True, get_alignment=True))

Index of centerstring (pythonic index): 4


AttributeError: 'DataFrame' object has no attribute 'to_text'